### Importing libraries

In [1]:
import openravepy as orpy
import trajoptpy
import json
import numpy as np
import trajoptpy.kin_utils as ku
from trajoptpy.check_traj import traj_is_safe
import time
import random 
import matplotlib.pyplot as plt
import pickle
from trajopt_util import *
from planning_util import *
from regression import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

FILENAME = 'data/denso_cartesian/'

### Setting the trajopt

In [2]:
np.set_printoptions(precision=4, suppress=True)

env = orpy.Environment()
env.StopSimulation()
env.Load("robots/weldbot_vs060.robot.xml")
#env.Load("../env/bookshelves.env.xml")
env.SetDefaultViewer()

True

In [3]:
env.Load("../env/bookshelves.env.xml")

True

### Setting the robot

In [4]:
robot = env.GetRobots()[0]
manip = robot.GetActiveManipulator()

In [5]:
init_transform = np.eye(4)
init_transform[:3,3] = [0.1, 0., .52712]
#init_transform[:3,3] = [2.6, 1, .92712]
robot.SetTransform(init_transform)
coeffs = [1,1,1,1,1,1]

In [6]:
dof = len(robot.GetActiveDOFValues())
init_joint0 = robot.GetActiveDOFValues()

### Set the IK Solver 

In [7]:
# IKFast
iktypes = [ orpy.IkParameterization.Type.Transform6D]
for iktype in iktypes:
    ikmodel = orpy.databases.inversekinematics.InverseKinematicsModel(robot, iktype=iktype)
    if not ikmodel.load():
        print('IKFast {0} solution not found'.format(iktype.name))
        
        print('Generating IKFast {0} solution. It will take couple of minutes ...'.format(iktype.name))
        t0 = time.time()
        ikmodel.autogenerate()
        duration = time.time() - t0
        print('IKFast %s generation took %.1f seconds' % (iktype.name, duration))
    else:
        print('IKFast {0} solution already generated'.format(iktype.name))


hmat_target = manip.GetEndEffectorTransform()

iktype = orpy.IkParameterization.Type.Transform6D
ikparam = orpy.IkParameterization(hmat_target, iktype)
opt = orpy.IkFilterOptions.CheckEnvCollisions
sols = manip.FindIKSolutions(ikparam,opt)

IKFast Transform6D solution already generated


### Construct a random problem 

In [8]:
def get_random_pose(env,limits, manip, box_size=0.04):
    robot = env.GetRobots()[0]
    is_success = False
    while is_success is not True:
        #Select which input space
        index = np.random.choice(limits.keys())
        x,y,z = generate_random_xyz(limits[index])
        
        #simple check for collision with a box
        if check_col_with_box(env,x,y,z,box_size):
            continue
            
        #do IK here
        xyz = [x,y,z]
        quat = [1,0,0,0] # wxyz
        pose = openravepy.matrixFromPose( np.r_[quat, xyz] )
        ikparam = orpy.IkParameterization(pose, iktype)
        opt = orpy.IkFilterOptions.CheckEnvCollisions
        sols = manip.FindIKSolutions(ikparam,opt)
        if len(sols) == 0:continue
                   
        is_success = True
        
    xyz =  np.array([x,y,z])     
    return sols, xyz

In [44]:
limits = dict()
limits[0] = [[0.47, 0.8],[-0.5, 0.5],[0.3,1.4]]

In [44]:
sols, xyz = get_random_pose(env, limits, manip)
print len(sols)

for sol in sols:
    robot.SetActiveDOFValues(sol)
    raw_input()

### Try planning 

### Build / Retrieve Dataset

In [56]:
def plan_to_joint(init_joint, target_joint, n_steps, init_type = 'given_traj', func=None):
    traj,_ = func.predict(init_joint, target_joint)     
    traj = traj.reshape(-1,dof)
    traj[0,:] = init_joint

    #Plan
    robot.SetActiveDOFValues(init_joint)
    request_traj = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='joint')
    request_traj = add_constraint(request_traj, 'joint', '', target_joint,-1)
    request_traj = set_init(request_traj, 'given_traj', traj)
    duration, result = run_opt(request_traj, env)    

    #Check traj result
    traj = result.GetTraj()
    traj_stat = check_traj(env,result, target_joint)

    return duration, result, traj, traj_stat

def build_database(x_init, y_init, num_traj, limits, n_steps = 30, func=None):
    num_plan = 0
    comp_times = []
    costs = []

    tic = time.time()
    while len(x_init) < num_traj:
        print('{}th Planning, {} trajectories in database'.format(num_plan,len(x_init)))
        #Get a random pose as target
        sols, xyz = get_random_pose(env,limits,manip)
        if sols is None:
            continue

        #plan to each solution
        for target_joint in sols:
            duration, result, traj, traj_stat = plan_to_joint(init_joint0, target_joint, n_steps, func=func)
            if traj_stat:
                print 'Planning is successfull!'
                x_init.append(xyz)
                y_init.append(traj.flatten())
                comp_times.append(duration)
                costs.append(result.GetCosts()[0][1])
            else:
                print('Fail to find good solution!')
            num_plan += 1
        
    #Store the result
    toc = time.time()
    total_time = toc-tic
    success_rate = num_traj*1.0/num_plan
    x_init = np.vstack(x_init)
    y_init = np.vstack(y_init)
    
    data = dict()
    data['x'] = x_init
    data['y'] = y_init
    data['total_time'] = total_time
    data['success_rate'] = success_rate
    data['comp_times'] = comp_times
    data['costs'] = costs
    return data

In [57]:
to_build_database = True
to_augment_database = False
num_traj = 2000
n_steps = 30
limits = dict()
limits[0] = [[0.47, 0.8],[-0.5, 0.5],[0.3,1.4]]

#build using straight-line warm-start
straight = Straight_Regressor(dof = dof, n_steps = n_steps)

if to_build_database:
    if to_augment_database:
        #load data
        data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
        x_init = list(data['x'])
        y_init = list(data['y'])
        num_traj += len(x_init)     
    else:
        x_init,y_init = [],[]
        
    data = build_database(x_init, y_init, num_traj,limits, n_steps = n_steps, func=straight)
    x_init = data['x']
    y_init = data['y']
    pickle.dump(data,open(FILENAME + 'data.pkl', 'wb') )
    print('Success_rate : {}, average costs:{}'.format(data['success_rate'], np.mean(data['costs'])))
else:
    #load data
    data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
    x_init = data['x']
    y_init = data['y']
    num_traj = len(x_init)
    print('Obtain {} trajectories'.format(num_traj))

0th Planning, 0 trajectories in database
Optimization is completed in 0.214353084564 s!
Planning is successfull!
Optimization is completed in 0.117298126221 s!
Planning is successfull!
Optimization is completed in 0.126477956772 s!
Planning is successfull!
Optimization is completed in 0.0118329524994 s!
Planning is successfull!
Optimization is completed in 0.0128488540649 s!
Planning is successfull!
Optimization is completed in 0.0124499797821 s!
Planning is successfull!
6th Planning, 6 trajectories in database
Optimization is completed in 0.452641010284 s!
Planning is successfull!
Optimization is completed in 0.217139005661 s!
Planning is successfull!
Optimization is completed in 0.210198879242 s!
Planning is successfull!
Optimization is completed in 0.0847489833832 s!
Planning is successfull!
Optimization is completed in 0.0493199825287 s!
Planning is successfull!
Optimization is completed in 0.107797861099 s!
Planning is successfull!
12th Planning, 12 trajectories in database
Optimi

### Reduce the data sample

### Apply Dimensionality Reduction to the data

In [58]:
from sklearn.decomposition import PCA
y_pca = PCA(n_components = 50)
y_pca.fit(y_init)
y_init_reduced = y_pca.transform(y_init)

### Check Database 

In [59]:
for i in range(5):
    index = np.random.randint(0,len(y_init))
    traj = y_init[index:index+1]
    traj = traj.reshape(-1,dof)
    plot_traj(env,traj)

### Reduce the data sample

In [60]:
def check_traj_pose(env,result, target, threshold = 0.01):
    #check trajectory with Cartesian target
    robot = env.GetRobots()[0]
    traj = result.GetTraj()
    
    #check for collision
    if traj_is_safe(traj, robot) is not True:
        print "There is a collision within the trajectory!"
        return False

    #check optimal solver status
    #if (result.GetStatus()[0] is not 0):
    #    print "Optimization is not converged!"
    #    return False

    robot.SetActiveDOFValues(traj[-1])
    T_arm = robot.GetLink('link6').GetTransform()
    pose = np.concatenate([T_arm[0:3,3].flatten(), openravepy.quatFromRotationMatrix(T_arm)])
    
    #check target for joint constraints
    if (np.linalg.norm(target - pose[:3]) > threshold):
        print target,pose
        print ('Pose is not reached!')
        return False


    #Otherwise, return True
    return True

### Build Function Approximator

In [61]:
dim_input = len(x_init[0])
is_load_regressor = False

straight = Straight_Regressor(dof = dof, n_steps=n_steps)
nn = NN_Regressor()
gpy = GPy_Regressor(dim_input = dim_input)
gpy_pca = GPy_Regressor(dim_input = dim_input, is_transform='PCA')
gpy_pca.pca = y_pca
bgmr = DP_GLM_Regressor()
bgmr_pca = DP_GLM_Regressor(is_transform='PCA')
bgmr_pca.pca = y_pca
if is_load_regressor:
    nn.load_from_file(FILENAME + 'nn')
    gpy.load_from_file(FILENAME + 'gpy')
    gpy_pca.load_from_file(FILENAME + 'gpy_pca')
    bgmr.load_from_file(FILENAME + 'bgmr')
    bgmr_pca.load_from_file(FILENAME + 'bgmr_pca')
else:
    print 'Planning for NN'
    nn.fit(x_init, y_init)
    nn.save_to_file(FILENAME + 'nn')

    print 'Planning for GPY'
    gpy.fit(x_init, y_init, num_restarts=10)
    gpy.save_to_file(FILENAME + 'gpy')
    
    print 'Planning for GPY PCA'
    gpy_pca.fit(x_init, y_init_reduced)
    gpy_pca.save_to_file(FILENAME + 'gpy_pca')

Planning for NN
Planning for GPY
Optimization restart 1/10, f = 504244.068387
Optimization restart 2/10, f = 504244.079234
Optimization restart 3/10, f = 504244.061175
Optimization restart 4/10, f = 504244.072505
Optimization restart 5/10, f = 504244.072185
Optimization restart 6/10, f = 504244.076848
Optimization restart 7/10, f = 504244.085942
Optimization restart 8/10, f = 504244.074871
Optimization restart 9/10, f = 504244.083234
Optimization restart 10/10, f = 504244.064686
Planning for GPY PCA
Optimization restart 1/10, f = 204147.463857
Optimization restart 2/10, f = 204147.466732
Optimization restart 3/10, f = 204147.478046
Optimization restart 4/10, f = 204147.50212
Optimization restart 5/10, f = 204147.464943
Optimization restart 6/10, f = 204147.46543
Optimization restart 7/10, f = 204147.468973
Optimization restart 8/10, f = 204147.464234
Optimization restart 9/10, f = 204147.472498
Optimization restart 10/10, f = 204147.47784


In [64]:
    print 'Planning for bgmr'
    bgmr.fit(x_init,y_init, n_components=30)
    bgmr.save_to_file(FILENAME + 'bgmr')
    
    print 'Planning for bgmr PCA'
    bgmr_pca.fit(x_init,y_init_reduced, n_components=30)
    bgmr_pca.save_to_file(FILENAME + 'bgmr_pca')

Planning for bgmr
Planning for bgmr PCA


#### Test Function Approximators


### Creating the test cases

In [65]:
create_test = True

if create_test:
    x_test = []
    target_joints = []
    num_test = 500
    for i in range(num_test):
        sols, xyz = get_random_pose(env,limits,manip,box_size = 0.04)
        if len(sols) == 0:
            continue
            
        target_joints += [sols[0]]
        x_test.append(xyz)
    
    x_test = np.vstack(x_test)
    target_joints = np.vstack(target_joints)
    
    data_test = dict()
    data_test['x_test'] = x_test
    data_test['target_joints'] = target_joints   
    test_file = open(FILENAME + 'data_test.pkl', 'wb')
    pickle.dump(data_test,test_file)
    test_file.close()
else:
    #load data
    test_file = open(FILENAME + 'data_test.pkl', 'rb')
    data_test = pickle.load(test_file)
    x_test = data_test['x_test']
    target_joints = data_test['target_joints']
    num_test = len(x_test)
    test_file.close()

### Testing the warmstart

In [66]:
method_names = ['STD    ', 'NN     ', 'GPR    ', 'GPR_PCA', 'BGMR', 'BGMR_PCA']
methods = [straight, nn, gpy, gpy_pca,  bgmr, bgmr_pca]
results = dict()
result,success, duration = dict(),dict(),dict()
for method in method_names:
    results[method] = dict()
    results[method]['costs'] = []
    results[method]['successes'] = []
    results[method]['comp_times'] = []
    results[method]['func_evals'] = []
    results[method]['qp_solves'] = []
    
num_test = 100
for it in range(num_test):
    print('{}th Planning'.format(it))
    
    #setting up the problem case
    #index = np.random.randint(0,len(x_test))
    index = it
    x_cur = x_test[index:index+1,:]
    target_joint = target_joints[index]
    robot.SetActiveDOFValues(init_joint0)
    
    #plan with various warmstarts
    for i,method in enumerate(methods):
        method_name = method_names[i]
        if isinstance(method, Straight_Regressor):
            traj,cov = method.predict(init_joint0, target_joint)
        else:
            traj,cov = method.predict(x_cur)
            
        if method.is_transform == 'PCA':
            traj = method.pca.inverse_transform(traj)
            
        traj = traj.reshape(-1,dof)
        traj[0,:] = init_joint0
        robot.SetActiveDOFValues(init_joint0)
        
        #request_traj = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='joint')
        #request_traj = add_constraint(request_traj, 'joint', '', target_joint,-1)
        #request_traj = set_init(request_traj, 'given_traj', traj)
        #duration[method_name], result[method_name] = run_opt(request_traj, env)    
        #success[method_name] = check_traj(env,result[method_name], target_joint)

        request_traj = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='pose')
        request_traj = set_init(request_traj, 'given_traj', traj)
        request_traj = add_constraint(request_traj, 'pose', 'link6', xyz,n_steps-1)
        duration[method_name], result[method_name]  = run_opt(request_traj, env)    
        #Check traj result
        success[method_name] = check_traj_pose(env,result[method_name], xyz)

    #Record the result
    for method_name in method_names:
        results[method_name]['costs'].append(result[method_name].GetCosts()[0][1])
        results[method_name]['func_evals'].append(result[method_name].GetNumFuncEval()[0])
        results[method_name]['qp_solves'].append(result[method_name].GetNumQPSolve()[0])
        results[method_name]['successes'].append(success[method_name])      
        results[method_name]['comp_times'].append(duration[method_name])

    print_result(results, method_names)

0th Planning
Optimization is completed in 0.208726882935 s!
Optimization is completed in 0.181233167648 s!
Optimization is completed in 0.318028211594 s!
Optimization is completed in 0.390466928482 s!
Optimization is completed in 0.206015110016 s!
Optimization is completed in 0.22700715065 s!
 Method 	| Success Rate 	| Conv. Time 	| Traj. Cost 	| Func. Evals 	| QP Solves
STD     	& 100.000 	& 0.21$\pm$0.00 	& 0.99$\pm$0.00 \\
NN      	& 100.000 	& 0.18$\pm$0.00 	& 0.98$\pm$0.00 \\
GPR     	& 100.000 	& 0.32$\pm$0.00 	& 0.16$\pm$0.00 \\
GPR_PCA 	& 100.000 	& 0.39$\pm$0.00 	& 0.16$\pm$0.00 \\
BGMR 	& 100.000 	& 0.21$\pm$0.00 	& 0.25$\pm$0.00 \\
BGMR_PCA 	& 100.000 	& 0.23$\pm$0.00 	& 0.91$\pm$0.00 \\
1th Planning
Optimization is completed in 0.179049968719 s!
Optimization is completed in 0.205707073212 s!
Optimization is completed in 0.265419960022 s!
Optimization is completed in 0.367152929306 s!
Optimization is completed in 0.289876937866 s!
Optimization is completed in 0.207242965698 

### Printing Results

In [67]:
print_result(results, method_names)

 Method 	| Success Rate 	| Conv. Time 	| Traj. Cost 	| Func. Evals 	| QP Solves
STD     	& 55.000 	& 0.16$\pm$0.09 	& 0.71$\pm$0.25 \\
NN      	& 52.000 	& 0.13$\pm$0.08 	& 0.72$\pm$0.21 \\
GPR     	& 48.000 	& 0.30$\pm$0.09 	& 0.21$\pm$0.15 \\
GPR_PCA 	& 49.000 	& 0.29$\pm$0.07 	& 0.26$\pm$0.20 \\
BGMR 	& 62.000 	& 0.16$\pm$0.10 	& 0.76$\pm$0.20 \\
BGMR_PCA 	& 62.000 	& 0.16$\pm$0.10 	& 0.75$\pm$0.20 \\


In [34]:
np.sum(results[method_names[0-1]]['successes'] or results[method_names[-6]]['successes'])

62

### Save Result

In [45]:
pickle.dump(results,open(FILENAME + 'result.pkl', 'wb') )

### Printing Results Tied to Standard Success

In [ ]:
print(' Method \t| Success Rate \t| Conv. Time \t| Traj. Cost \t| Func. Evals \t| QP Solves')
standard_successes = np.array(results['STD    ']['successes'])

for method in method_names:
    successes = np.array(results[method]['successes'])[standard_successes]
    success = np.count_nonzero(successes)
    comp_times = np.array(results[method]['comp_times'])[standard_successes][successes]
    costs = np.array(results[method]['costs'])[standard_successes][successes]
    func_evals = np.array(results[method]['func_evals'])[standard_successes][successes]
    qp_solves = np.array(results[method]['qp_solves'])[standard_successes][successes]
    
    print('{0}: \t {1:.3f} \t {2:.2f} \t {3:.3f} \t {4:.3f} \t {5:.3f}'.format(method, success*1.0/len(successes), np.sum(comp_times)/success, np.sum(costs)/success, 1.0*np.sum(func_evals)/success, 1.0*np.sum(qp_solves)/success ))
    